In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests 

In [2]:
url="http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_1"


In [126]:
def getInfo(url):
    req = requests.get(url)
    bs = BeautifulSoup(req.text, 'html.parser')



    #!!!!!!!!!!!!!!!!!!!
    # region = 
    region='муниципальный'
    #!!!!!!!!!!!!!!!!!!!



    # str_url = urllib.parse.unquote(url.rsplit('/',1)[1])
    str_url = url.rsplit('/',1)[1]
    lst = str_url.split("_")
    # 2 - номер маршрута
    nmbr = lst[-1]
    # 1 - тип транспорта
    tp_ts = lst[-2]
    # rt_nmbr = url.rsplit("_",1)[1]

    def getTable(table_rows):
        l = []
        for tr in table_rows:
            try:
                td = tr.find_all('td')
                row = [tr.text for tr in td]
                l.append(row)
            except:
                pass
        #
        return l
    #
    # 2 - содержание
    # 3 - маршрут следования
    # 5 - оп в прямом направлении
    # 6 - оп в обратном направлении
    # 7 - особенности работы
    # 8 - расписание (две версии, для регионов!!)

    all_tables = bs.find_all("table")
    l2 = getTable(all_tables[2])
    l3 = getTable(all_tables[3])
    l5 = getTable(all_tables[5])
    l6 = getTable(all_tables[6])
    l7 = getTable(all_tables[7])
    l8 = getTable(all_tables[8])

    ####
    # 3 - наименование маршрута
    str1 = l2[0][0]
    str1 = str1.replace("\n\n","\n")
    rt_nm = str1.split("\n")[1][2:].split("№")[1]
    rt_nm = rt_nm.split('"', 1)[1][:-1]

    ####
    # 4 -маршрут следования
    route_frw = l3[0][1]
    route_bckw = l3[1][1]
    ####

    ####
    # 5,6 - список оп
    def stops(l):
    #
        str_all=''
        for row in l[2:]:
            rw = []
            for i in row:
                i = i.replace("\n","")
                rw.append(i)
            str1 = rw[1]
            str_all = str_all + "_" + str1
        #
        str_all = str_all[1:]
        return str_all
    # 
    stops_frw = stops(l5)
    stops_bckw = stops(l6)
    ####

    ####
    # 7) Режим работы
    # 8) Интервалы движения
    # 9) Выпуск на линию
    # 10) Время в пути
    # 11) Протяжённость маршрута
    # 12) Обслуживающие предприятия
    # 13) Подвижной состав
    # 14) Стоимость проезда
    # 15) Место отстоя 

    nlst = []
    for row in l7:
        tmplst = []
        for j in row:
            j = j.replace("\n","")
            tmplst.append(j)
        nlst.append(tmplst)
    # 

    work_time = nlst[0][1]
    intervs = nlst[1][1]
    amnt_mch = nlst[2][1]
    duration = nlst[3][1]
    length = nlst[4][1]
    orgnztn = nlst[5][1]
    podv_sost = nlst[6][1]
    payment = nlst[7][1]
    place_rest = nlst[8][1]
    ####


    # get all to list
    lst_all = [tp_ts,region,nmbr,rt_nm,route_frw,route_bckw,stops_frw,stops_bckw,
                    work_time,intervs,amnt_mch,duration,length,orgnztn,podv_sost,payment,place_rest,url]
    #
    return lst_all

In [127]:
lst1 = getInfo(url)

In [130]:
lst1[0]

'%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81'

In [98]:
# reg = re.compile('[^0-9-]')
# one_line = df['data'][1].split(" мин")[0]
# if "раб" in one_line:
#     one_line2 = one_line.split(":")[1]
#     one_line2 = reg.sub('', one_line2)
#     print(one_line2)

8-20


In [ ]:
# проверять table title
# "href"

In [13]:
url="http://www.samaratrans.info/wiki/index.php/%D0%97%D0%B0%D0%B3%D0%BB%D0%B0%D0%B2%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0"


In [14]:
req = requests.get(url)
bs = BeautifulSoup(req.text, 'html.parser')

In [35]:
def getLinks(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
#     html_page = urllib2.urlopen(url)
#     soup = BeautifulSoup(html_page)
    links = []

    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))

    return links

In [36]:
links = getLinks(url)

In [34]:
import re

In [42]:
new_links = []

for elem in links:
#     if "http://www.samaratrans.info/wiki/index.php/Самара_" in elem:
    if "маршрут" in elem:
        new_links.append(elem)

In [45]:
new_links

['http://www.samaratrans.info/wiki/index.php/Самара_автобус_маршруты',
 'http://www.samaratrans.info/wiki/index.php/Самара_автобус_коммерческий_маршруты',
 'http://www.samaratrans.info/wiki/index.php/Самара_автобус_сезонный_маршруты',
 'http://www.samaratrans.info/wiki/index.php/Самара_автобус_специальный_маршруты',
 'http://www.samaratrans.info/wiki/index.php/Самара_трамвай_маршруты',
 'http://www.samaratrans.info/wiki/index.php/Самара_троллейбус_маршруты']

In [46]:
url = new_links[0]

In [49]:
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [83]:
links=[]
stfa = soup.find_all("table")[-3].find_all('a')
for link in stfa:
    lsh = link.get('href')
    lsh = 'http://www.samaratrans.info' + lsh
    links.append(lsh)
# 